In [1]:
import sys, os 
import gym 
import gymnasium
import numpy as np
import gym_sudoku
import stable_baselines3

In [2]:
from gym_sudoku.envs import SudokuEnv

## Reinforcement Learning

* [Stable Baselines 3 (Pytorch)](https://github.com/DLR-RM/stable-baselines3)

In [3]:
# Trying to make it backwards compatible
env = gym.make("Sudoku-v0")
#env = gymnasium.make('Sudoku-v0', env=SudokuEnv())
#env= gym.make('Sudoku-v0',apply_api_compatibility=True)
# env.reset()

In [4]:
env.reset()

array([[0, 7, 0, 0, 0, 0, 0, 9, 0],
       [8, 6, 0, 0, 3, 5, 0, 1, 0],
       [9, 3, 0, 4, 0, 2, 5, 6, 8],
       [0, 0, 0, 0, 5, 1, 9, 0, 6],
       [5, 0, 9, 6, 4, 3, 7, 2, 0],
       [0, 0, 0, 2, 0, 8, 0, 0, 5],
       [7, 2, 8, 0, 0, 9, 0, 5, 0],
       [0, 5, 0, 8, 0, 0, 0, 7, 0],
       [1, 0, 4, 0, 6, 0, 8, 0, 0]])

In [5]:
grid = env.reset()

In [6]:
flatgrid = grid.flatten()

In [7]:
flatgrid.reshape(9,9)

array([[0, 7, 0, 0, 0, 0, 0, 9, 0],
       [8, 6, 0, 0, 3, 5, 0, 1, 0],
       [9, 3, 0, 4, 0, 2, 5, 6, 8],
       [0, 0, 0, 0, 5, 1, 9, 0, 6],
       [5, 0, 9, 6, 4, 3, 7, 2, 0],
       [0, 0, 0, 2, 0, 8, 0, 0, 5],
       [7, 2, 8, 0, 0, 9, 0, 5, 0],
       [0, 5, 0, 8, 0, 0, 0, 7, 0],
       [1, 0, 4, 0, 6, 0, 8, 0, 0]])

In [8]:
env.render()

070 | 000 | 090
860 | 035 | 010
930 | 402 | 568
---------------
000 | 051 | 906
509 | 643 | 720
000 | 208 | 005
---------------
728 | 009 | 050
050 | 800 | 070
104 | 060 | 800




In [9]:
gym.__version__

'0.21.0'

### Simple Deep Q
stable_baselines3.dqn

In [10]:
from stable_baselines3 import DQN, PPO, A2C
from stable_baselines3.dqn import MlpPolicy, CnnPolicy

In [11]:
from gym.spaces import Discrete, Tuple

In [14]:
model = DQN("MlpPolicy",env, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [15]:
import time
start = time.perf_counter()
model.learn(total_timesteps=10_000_000)
length = time.perf_counter() - start 

KeyboardInterrupt: 

In [23]:
100_000

100000

In [27]:
obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action.item())
    print(action)

env.render()

696
696
696
696
696
696
696
233
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
354
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
560
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
332
696
696
137
658
696
696
696
696
377
696
696
696
31
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
530
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
720
168
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
696
6

In [28]:
env.render()

000 | 510 | 406
360 | 257 | 300
759 | 836 | 297
---------------
721 | 009 | 035
930 | 179 | 902
598 | 620 | 414
---------------
973 | 790 | 003
002 | 300 | 794
941 | 543 | 303




In [22]:
obs

Box(1, 9, (9, 9), int64)

In [ ]:
obs